In [1]:
import pandas as pd

In [13]:
# Load phases dataframe
parse_dates = ["start", "end", "travel_date", "latest_res_dt"]
df = pd.read_csv("../model_data.csv", parse_dates=parse_dates)
df

,start,start_loc,count,end,end_loc,capacity,line,travel_date,latest_res_dt
0,2021-03-21 06:34:30,ZUE,1.0,2021-03-21 07:28:00,BS,15.0,IC 3,2021-03-21,2021-03-21 06:34:30
1,2021-03-26 05:08:30,CH,2.0,2021-03-26 06:22:18,ZUE,9.0,IC 3,2021-03-26,2021-03-26 05:08:30
2,2021-03-26 06:34:30,ZUE,1.0,2021-03-26 07:28:00,BS,9.0,IC 3,2021-03-26,2021-03-26 06:34:30
3,2021-03-27 05:08:30,CH,1.0,2021-03-27 06:22:18,ZUE,9.0,IC 3,2021-03-27,2021-03-27 05:08:30
4,2021-04-24 05:08:30,CH,1.0,2021-04-24 06:22:18,ZUE,9.0,IC 3,2021-04-24,2021-04-24 05:08:30
...,...,...,...,...,...,...,...,...,...
115769,2021-10-17 15:47:30,LQ,7.0,2021-10-17 15:56:24,SA,37.0,IC 3,2021-10-17,2021-10-17 15:47:30
115770,2021-10-17 15:56:24,SA,8.0,2021-10-17 16:53:00,ZUE,37.0,IC 3,2021-10-17,2021-10-17 15:56:24
115771,2021-10-24 15:37:30,CH,7.0,2021-10-24 15:47:30,LQ,35.0,IC 3,2021-10-24,2021-10-24 15:37:30
115772,2021-10-24 15:47:30,LQ,11.0,2021-10-24 15:56:24,SA,35.0,IC 3,2021-10-24,2021-10-24 15:47:30


In [14]:
# train station coordinates data
df_coordinates = pd.read_csv('../data_preprocessed/dienststellen.csv')
df_coordinates = df_coordinates[["abk_bahnhof", "lat", "lon"]]
df_coordinates = df_coordinates[df_coordinates['abk_bahnhof'].notna()]
display(df_coordinates.tail(1))

/Users/oli/miniforge3/envs/tensorflow/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,abk_bahnhof,lat,lon
47113,BRIA,47.260083,7.942674


In [15]:
# full join for start train station 
df = pd.merge(df, df_coordinates, left_on='start_loc', right_on='abk_bahnhof')
df = df.drop(columns=['abk_bahnhof']).rename(columns={"lat": "lat_from", "lon": "lon_from"})

# full join for destination
df = pd.merge(df, df_coordinates, left_on='end_loc', right_on='abk_bahnhof')
df = df.drop(columns=["abk_bahnhof", "start_loc", "end_loc"]).rename(columns={"lat": "lat_to", "lon": "lon_to"})
df

,start,count,end,capacity,line,travel_date,latest_res_dt,lat_from,lon_from,lat_to,lon_to
0,2021-03-21 06:34:30,1.0,2021-03-21 07:28:00,15.0,IC 3,2021-03-21,2021-03-21 06:34:30,47.378177,8.540212,47.547412,7.589563
1,2021-03-26 06:34:30,1.0,2021-03-26 07:28:00,9.0,IC 3,2021-03-26,2021-03-26 06:34:30,47.378177,8.540212,47.547412,7.589563
2,2021-05-01 06:34:30,2.0,2021-05-01 07:28:00,9.0,IC 3,2021-05-01,2021-05-01 06:34:30,47.378177,8.540212,47.547412,7.589563
3,2021-05-08 06:34:30,1.0,2021-05-08 07:28:00,9.0,IC 3,2021-05-08,2021-05-08 06:34:30,47.378177,8.540212,47.547412,7.589563
4,2021-05-30 06:34:30,2.0,2021-05-30 07:28:00,13.0,IC 3,2021-05-30,2021-05-30 06:34:30,47.378177,8.540212,47.547412,7.589563
...,...,...,...,...,...,...,...,...,...,...,...
115769,2021-06-11 05:30:30,1.0,2021-06-11 05:40:30,12.0,IC 6/61,2021-06-11,2021-06-11 05:30:30,46.754853,7.629606,46.873354,7.559442
115770,2021-08-20 05:55:48,2.0,2021-08-20 06:11:42,15.0,IC 8,2021-08-20,2021-08-20 05:55:48,46.294029,7.881465,46.588908,7.651418
115771,2021-08-21 05:55:48,2.0,2021-08-21 06:14:12,9.0,IC 8,2021-08-21,2021-08-21 05:55:48,46.294029,7.881465,46.588908,7.651418
115772,2021-09-03 05:55:48,1.0,2021-09-03 06:11:42,9.0,IC 8,2021-09-03,2021-09-03 05:55:48,46.294029,7.881465,46.588908,7.651418


In [ ]:
def get_time_representation(ts):
    return (
        ts.year*10000000000 +
        ts.month * 100000000 +
        ts.day * 1000000 +
        ts.hour*10000 +
        ts.minute*100 +
        ts.second
    )

In [16]:
df_data = df.drop(["count", "travel_date"], axis=1)
df_data

,start,end,capacity,line,latest_res_dt,lat_from,lon_from,lat_to,lon_to
0,2021-03-21 06:34:30,2021-03-21 07:28:00,15.0,IC 3,2021-03-21 06:34:30,47.378177,8.540212,47.547412,7.589563
1,2021-03-26 06:34:30,2021-03-26 07:28:00,9.0,IC 3,2021-03-26 06:34:30,47.378177,8.540212,47.547412,7.589563
2,2021-05-01 06:34:30,2021-05-01 07:28:00,9.0,IC 3,2021-05-01 06:34:30,47.378177,8.540212,47.547412,7.589563
3,2021-05-08 06:34:30,2021-05-08 07:28:00,9.0,IC 3,2021-05-08 06:34:30,47.378177,8.540212,47.547412,7.589563
4,2021-05-30 06:34:30,2021-05-30 07:28:00,13.0,IC 3,2021-05-30 06:34:30,47.378177,8.540212,47.547412,7.589563
...,...,...,...,...,...,...,...,...,...
115769,2021-06-11 05:30:30,2021-06-11 05:40:30,12.0,IC 6/61,2021-06-11 05:30:30,46.754853,7.629606,46.873354,7.559442
115770,2021-08-20 05:55:48,2021-08-20 06:11:42,15.0,IC 8,2021-08-20 05:55:48,46.294029,7.881465,46.588908,7.651418
115771,2021-08-21 05:55:48,2021-08-21 06:14:12,9.0,IC 8,2021-08-21 05:55:48,46.294029,7.881465,46.588908,7.651418
115772,2021-09-03 05:55:48,2021-09-03 06:11:42,9.0,IC 8,2021-09-03 05:55:48,46.294029,7.881465,46.588908,7.651418
